In [26]:
import geopandas
import pandas as pd
import pgeocode
import plotly.graph_objects as go
nomi = pgeocode.Nominatim('ca')

In [34]:
distances = pd.read_csv('data/Connected member addresses for network map November 2022 - Sheet1.csv', header=None, 
                   names=['Member', 'Address', 'City', 'Postal Code'])
distances.iloc[56]['Postal Code'] = 'T0L 0Z0' # Someone entered this wihout a space
distances.drop([36, 38], axis=0, inplace=True) # Centers in Germany and France
distances

,Member,Address,City,Postal Code
0,Alberta Conference SDA Church,5816 Hwy 2A,Lacombe,T4L 2G5
1,Alberta Health Services,48 Quarry Park Blvd. SE,Calgary,T2C 5P2
2,Alberta Innovates,"1500, 10104 - 103 Avenue",Edmonton,T5J 0H8
3,Alberta University of the Arts,1407 - 14th Avenue NW,Calgary,T2N 4R3
4,Aspen View School Division,3600 - 48 Avenue,Athabasca,T9S 1M8
...,...,...,...,...
94,University of Lethbridge,4401 University Drive,Lethbridge,T1K 3M4
95,Wetaskiwin Regional Public Schools,5704 - 51 Street,Wetaskiwin,T9A 3G1
96,Whipcord Ltd,3528 30 Street North,Lethbridge,T1H 6Z4
97,Wild Rose School Division No.66,4912 - 43 Street,Rocky Mountain House,T4T 1P4


In [35]:
lats = []
lons = []
for i in distances['Postal Code']:
    location = nomi.query_postal_code(i)
    lats.append(location['latitude'])
    lons.append(location['longitude'])
distances['lat'] = lats
distances['lon'] = lons
distances['geometry'] = geopandas.points_from_xy(distances['lat'], distances['lon'])
distances

,Member,Address,City,Postal Code,lat,lon,geometry
0,Alberta Conference SDA Church,5816 Hwy 2A,Lacombe,T4L 2G5,52.4668,-113.7353,POINT (52.467 -113.735)
1,Alberta Health Services,48 Quarry Park Blvd. SE,Calgary,T2C 5P2,50.9870,-113.9634,POINT (50.987 -113.963)
2,Alberta Innovates,"1500, 10104 - 103 Avenue",Edmonton,T5J 0H8,53.5428,-113.4974,POINT (53.543 -113.497)
3,Alberta University of the Arts,1407 - 14th Avenue NW,Calgary,T2N 4R3,51.0621,-114.1159,POINT (51.062 -114.116)
4,Aspen View School Division,3600 - 48 Avenue,Athabasca,T9S 1M8,54.7169,-113.2854,POINT (54.717 -113.285)
...,...,...,...,...,...,...,...
94,University of Lethbridge,4401 University Drive,Lethbridge,T1K 3M4,49.6511,-112.8351,POINT (49.651 -112.835)
95,Wetaskiwin Regional Public Schools,5704 - 51 Street,Wetaskiwin,T9A 3G1,52.9668,-113.3687,POINT (52.967 -113.369)
96,Whipcord Ltd,3528 30 Street North,Lethbridge,T1H 6Z4,49.7000,-112.8186,POINT (49.700 -112.819)
97,Wild Rose School Division No.66,4912 - 43 Street,Rocky Mountain House,T4T 1P4,52.3668,-114.9188,POINT (52.367 -114.919)


In [29]:
fig0 = go.Figure(data=go.Scattergeo(
    lat = distances['lat'],
    lon = distances['lon'],
    text = distances['Member']
)
               )

fig0.update_layout(
    geo = dict(
        scope = 'north america',
        showland = True,
        #landcolor = "rgb(212, 212, 212)",
        landcolor = "rgb(255, 255, 255)",
        subunitcolor = "rgb(0, 0, 0)",
        countrycolor = "rgb(255, 255, 255)",
        showlakes = False,
        lakecolor = "rgb(255, 255, 255)",
        showsubunits = True,
        showcountries = True,
        resolution = 50,
        projection = dict(
            type = 'conic conformal',
            rotation_lon = -100
        ),
        lonaxis = dict(
            showgrid = True,
            gridwidth = 0.5,
            range= [ -140.0, -55.0 ],
            dtick = 5
        ),
        lataxis = dict (
            showgrid = True,
            gridwidth = 0.5,
            range= [ 20.0, 60.0 ],
            dtick = 5
        )
    ),
    title='Cybera Member locations',
    height=600,
    width=600,
    margin={"r":0,"t":0,"l":0,"b":0}
)
fig0.update_geos(lataxis_range=[48,61],
               lonaxis_range=[-120,-110])
fig0.show()
#fig.write_html('membermap.html')

In [6]:
#!pip install kaleido
fig0.write_image('membermap.svg')

## Grouping dots together

In [36]:
try:
    import haversine as hs
except:
    !pip install --user haversine
    import haversine as hs

# Calculate the distance between each member
distances_list = []
for i in range(len(distances)):
    for j in range(i+1, len(distances)):
        distances_list.append([distances.iloc[i]['Member'], distances.iloc[j]['Member'], 
                          hs.haversine((distances.iloc[i]['lat'], distances.iloc[i]['lon']), 
                                       (distances.iloc[j]['lat'], distances.iloc[j]['lon']))])
distances_between = pd.DataFrame(distances_list, columns=['Member1', 'Member2', 'Distance'])
distances_between

,Member1,Member2,Distance
0,Alberta Conference SDA Church,Alberta Health Services,165.294534
1,Alberta Conference SDA Church,Alberta Innovates,120.699983
2,Alberta Conference SDA Church,Alberta University of the Arts,158.375930
3,Alberta Conference SDA Church,Aspen View School Division,251.954297
4,Alberta Conference SDA Church,Athabasca University,251.954297
...,...,...,...
4651,Wetaskiwin Regional Public Schools,Wild Rose School Division No.66,124.002373
4652,Wetaskiwin Regional Public Schools,Wolf Creek Regional Division No.32,35.274654
4653,Whipcord Ltd,Wild Rose School Division No.66,330.874364
4654,Whipcord Ltd,Wolf Creek Regional Division No.32,335.243113


In [73]:
# this will only work if they are sorted, but it's not great
# we should elimilate one of the two members in a pair that is too close
# but there are lots of groups that are three or more members close together

separation_distance = 150

previous_row = None
rows_to_remove = []
for row in distances.sort_values(by='lat').itertuples():
    #print(row.Member, row.lat, row.lon)
    if previous_row is not None:
        #print(previous_row.Member, row.Member, 
        d = hs.haversine((previous_row.lat, previous_row.lon), (row.lat, row.lon))
        if d < separation_distance:
            #print('remove row', row.Index)
            rows_to_remove.append(row.Index)
    previous_row = row
filtered_distances = distances.drop(rows_to_remove, axis=0)
import folium
#from folium.plugins import FastMarkerCluster
median_latitude = filtered_distances['lat'].median()
median_longitude = filtered_distances['lon'].median()
new_map = folium.Map(location=[median_latitude, median_longitude], zoom_start=6, tiles='stamen terrain')
#new_map.add_child(FastMarkerCluster(filtered_distances[['lat','lon']].values.tolist()))
for i in filtered_distances.itertuples():
    folium.Marker([i.lat, i.lon], popup=i.Member).add_to(new_map)
display(new_map)

In [16]:
len(distances_list[distances_list['Distance'] < 20].sort_values('Distance')['Member1'].unique())

59

In [17]:
# map the members that are close to each other
distances_list['Member 1 Lat'] = distances_list['Member1'].map(distances_list.set_index('Member')['lat'])
distances_list['Member 1 Lon'] = distances_list['Member1'].map(distances_list.set_index('Member')['lon'])
distances_list

,Member1,Member2,Distance,Member 1 Lat,Member 1 Lon
0,Alberta Conference SDA Church,Alberta Health Services,165.294534,52.4668,-113.7353
1,Alberta Conference SDA Church,Alberta Innovates,120.699983,52.4668,-113.7353
2,Alberta Conference SDA Church,Alberta University of the Arts,158.375930,52.4668,-113.7353
3,Alberta Conference SDA Church,Aspen View School Division,251.954297,52.4668,-113.7353
4,Alberta Conference SDA Church,Athabasca University,251.954297,52.4668,-113.7353
...,...,...,...,...,...
4651,Wetaskiwin Regional Public Schools,Wild Rose School Division No.66,124.002373,52.9668,-113.3687
4652,Wetaskiwin Regional Public Schools,Wolf Creek Regional Division No.32,35.274654,52.9668,-113.3687
4653,Whipcord Ltd,Wild Rose School Division No.66,330.874364,49.7000,-112.8186
4654,Whipcord Ltd,Wolf Creek Regional Division No.32,335.243113,49.7000,-112.8186


In [25]:
distances_list[(distances_list['Member1']=='Alberta Conference SDA Church') & (distances_list['Member2']=='Alberta Conference SDA Church')]

,Member1,Member2,Distance,Member 1 Lat,Member 1 Lon
0,Alberta Conference SDA Church,Alberta Health Services,165.294534,52.4668,-113.7353
1,Alberta Conference SDA Church,Alberta Innovates,120.699983,52.4668,-113.7353
2,Alberta Conference SDA Church,Alberta University of the Arts,158.375930,52.4668,-113.7353
3,Alberta Conference SDA Church,Aspen View School Division,251.954297,52.4668,-113.7353
4,Alberta Conference SDA Church,Athabasca University,251.954297,52.4668,-113.7353
...,...,...,...,...,...
91,Alberta Conference SDA Church,University of Lethbridge,319.343385,52.4668,-113.7353
92,Alberta Conference SDA Church,Wetaskiwin Regional Public Schools,60.834276,52.4668,-113.7353
93,Alberta Conference SDA Church,Whipcord Ltd,314.240368,52.4668,-113.7353
94,Alberta Conference SDA Church,Wild Rose School Division No.66,81.029767,52.4668,-113.7353


In [23]:
fig0 = go.Figure(data=go.Scattergeo(
    lat = distances_list['Member 1 Lat'],
    lon = distances_list['Member 1 Lon'],
    text = distances_list['Member1']
)
               )

fig0.update_layout(
    geo = dict(
        scope = 'north america',
        showland = True,
        #landcolor = "rgb(212, 212, 212)",
        landcolor = "rgb(255, 255, 255)",
        subunitcolor = "rgb(0, 0, 0)",
        countrycolor = "rgb(255, 255, 255)",
        showlakes = False,
        lakecolor = "rgb(255, 255, 255)",
        showsubunits = True,
        showcountries = True,
        resolution = 50,
        projection = dict(
            type = 'conic conformal',
            rotation_lon = -100
        ),
        lonaxis = dict(
            showgrid = True,
            gridwidth = 0.5,
            range= [ -140.0, -55.0 ],
            dtick = 5
        ),
        lataxis = dict (
            showgrid = True,
            gridwidth = 0.5,
            range= [ 20.0, 60.0 ],
            dtick = 5
        )
    ),
    title='Cybera Member locations',
    height=600,
    width=600,
    margin={"r":0,"t":0,"l":0,"b":0}
)
fig0.update_geos(lataxis_range=[48,61],
               lonaxis_range=[-120,-110])
fig0.show()

## Finding scale distances on a 4x8 sheet of plywood

In [ ]:
x_range = [distances_list['lon'].max(), distances_list['lon'].min()]
y_range = [distances_list['lat'].max(), distances_list['lat'].min()]
distances_list['x'] = (distances_list['lon'] - x_range[1]) / (x_range[0] - x_range[1])
distances_list['y'] = (distances_list['lat'] - y_range[1]) / (y_range[0] - y_range[1])
distances_list['x inches'] = distances_list['x'] * 46
distances_list['y inches'] = distances_list['y'] * 46
distances_list

,Member,Address,City,Postal Code,lat,lon,geometry,x,y,x inches,y inches
0,Alberta Conference SDA Church,5816 Hwy 2A,Lacombe,T4L 2G5,52.4668,-113.7353,POINT (52.467 -113.735),0.616806,0.356906,28.373063,16.417660
1,Alberta Health Services,48 Quarry Park Blvd. SE,Calgary,T2C 5P2,50.9870,-113.9634,POINT (50.987 -113.963),0.589619,0.169333,27.122455,7.789307
2,Alberta Innovates,"1500, 10104 - 103 Avenue",Edmonton,T5J 0H8,53.5428,-113.4974,POINT (53.543 -113.497),0.645161,0.493295,29.677402,22.691553
3,Alberta University of the Arts,1407 - 14th Avenue NW,Calgary,T2N 4R3,51.0621,-114.1159,POINT (51.062 -114.116),0.571442,0.178852,26.286341,8.227197
4,Aspen View School Division,3600 - 48 Avenue,Athabasca,T9S 1M8,54.7169,-113.2854,POINT (54.717 -113.285),0.670429,0.642118,30.839738,29.537444
...,...,...,...,...,...,...,...,...,...,...,...
94,University of Lethbridge,4401 University Drive,Lethbridge,T1K 3M4,49.6511,-112.8351,POINT (49.651 -112.835),0.724100,0.000000,33.308605,0.000000
95,Wetaskiwin Regional Public Schools,5704 - 51 Street,Wetaskiwin,T9A 3G1,52.9668,-113.3687,POINT (52.967 -113.369),0.660501,0.420283,30.383027,19.333038
96,Whipcord Ltd,3528 30 Street North,Lethbridge,T1H 6Z4,49.7000,-112.8186,POINT (49.700 -112.819),0.726067,0.006198,33.399070,0.285124
97,Wild Rose School Division No.66,4912 - 43 Street,Rocky Mountain House,T4T 1P4,52.3668,-114.9188,POINT (52.367 -114.919),0.475745,0.344230,21.884267,15.834584


In [ ]:
import plotly.express as px
fig2 = px.scatter(distances_list, x='x inches', y='y inches', hover_name='Member', hover_data=['Address', 'City', 'Postal Code'], height=800, width=400)
fig2.update_xaxes(range=[0, 48])
fig2.update_yaxes(range=[0, 96])
fig2.show()
fig0.show()